In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

import Resnet50 as res
from utils import progress_bar

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


In [3]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
print('==> Building model..')
net = res.resnet50()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [5]:
#print('==> Resuming from checkpoint..')
#assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#checkpoint = torch.load('./checkpoint/ckpt.pth')
#net.load_state_dict(checkpoint['net'])
#best_acc = checkpoint['acc']
#start_epoch = checkpoint['epoch']

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


In [7]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [8]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    # Save checkpoint.    
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [9]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0


 [================================================================>]  Step: 1s668ms | Tot: 1m23s | Loss: 2.886 | Acc: 16.940% (8470/5000 391/391 =================================>.............]  Step: 205ms | Tot: 1m5s | Loss: 3.096 | Acc: 15.693% (6267/3993 312/391 =========================>............]  Step: 208ms | Tot: 1m6s | Loss: 3.086 | Acc: 15.761% (6355/4032 315/391 ====================================>............]  Step: 206ms | Tot: 1m6s | Loss: 3.083 | Acc: 15.788% (6386/4044 316/391 ============================================>............]  Step: 209ms | Tot: 1m6s | Loss: 3.077 | Acc: 15.799% (6431/4070 318/391 ===========================>...........]  Step: 207ms | Tot: 1m7s | Loss: 3.068 | Acc: 15.825% (6502/4108 321/391 =============================================>..........]  Step: 208ms | Tot: 1m8s | Loss: 3.053 | Acc: 15.867% (6621/4172 326/391 =============================================>..........]  Step: 209ms | Tot: 1m8s | Loss: 3.047 | Acc: 15.916% (6682/4

 [================================================================>]  Step: 145ms | Tot: 1m22s | Loss: 1.848 | Acc: 30.402% (15201/5000 391/391 ==============================================>............]  Step: 208ms | Tot: 1m6s | Loss: 1.876 | Acc: 29.197% (11847/4057 317/391 ========>............]  Step: 210ms | Tot: 1m7s | Loss: 1.875 | Acc: 29.231% (11898/4070 318/391 ====================================================>...........]  Step: 206ms | Tot: 1m7s | Loss: 1.873 | Acc: 29.279% (12030/4108 321/391 ======================>...........]  Step: 208ms | Tot: 1m8s | Loss: 1.872 | Acc: 29.326% (12162/4147 324/391 ==============================================>..........]  Step: 209ms | Tot: 1m8s | Loss: 1.871 | Acc: 29.376% (12258/4172 326/391 =====================================================>..........]  Step: 212ms | Tot: 1m9s | Loss: 1.871 | Acc: 29.401% (12306/4185 327/391 ================================>..........]  Step: 206ms | Tot: 1m9s | Loss: 1.870 | Acc: 29.452% (1

 [================================================================>]  Step: 139ms | Tot: 1m23s | Loss: 1.638 | Acc: 38.992% (19496/5000 391/391 =======================================>....]  Step: 211ms | Tot: 1m17s | Loss: 1.643 | Acc: 38.785% (18021/4646 363/391 .]  Step: 210ms | Tot: 1m17s | Loss: 1.642 | Acc: 38.797% (18126/4672 365/391 =======>...]  Step: 208ms | Tot: 1m18s | Loss: 1.641 | Acc: 38.844% (18347/4723 369/391 =================================================>...]  Step: 206ms | Tot: 1m19s | Loss: 1.641 | Acc: 38.886% (18516/4761 372/391 =====================================>..]  Step: 209ms | Tot: 1m19s | Loss: 1.640 | Acc: 38.916% (18630/4787 374/391 =======================================================>..]  Step: 210ms | Tot: 1m20s | Loss: 1.640 | Acc: 38.919% (18731/4812 376/391 ============================================================>..]  Step: 210ms | Tot: 1m20s | Loss: 1.640 | Acc: 38.924% (18833/4838 378/391 ===============================================

 [================================================================>]  Step: 141ms | Tot: 1m22s | Loss: 1.467 | Acc: 45.688% (22844/5000 391/391 1 ============================>................]  Step: 209ms | Tot: 1m1s | Loss: 1.486 | Acc: 44.975% (16810/3737 292/391 ===============================================>................]  Step: 211ms | Tot: 1m1s | Loss: 1.485 | Acc: 44.967% (16922/3763 294/391 ============================================>...............]  Step: 207ms | Tot: 1m2s | Loss: 1.485 | Acc: 44.983% (17043/3788 296/391 ===============================>...............]  Step: 210ms | Tot: 1m2s | Loss: 1.485 | Acc: 44.974% (17155/3814 298/391 =================================>...............]  Step: 210ms | Tot: 1m3s | Loss: 1.485 | Acc: 44.964% (17266/3840 300/391 ==============================>..............]  Step: 210ms | Tot: 1m3s | Loss: 1.485 | Acc: 44.958% (17379/3865 302/391 =============================================>..............]  Step: 208ms | Tot: 1m3s |

...

 [================================================================>]  Step: 142ms | Tot: 1m20s | Loss: 0.001 | Acc: 99.982% (49991/5000 391/391 1  ==================================>..................]  Step: 216ms | Tot: 58s401ms | Loss: 0.001 | Acc: 99.983% (36090/3609 282/391 ===========================>.................]  Step: 208ms | Tot: 59s230ms | Loss: 0.001 | Acc: 99.984% (36602/3660 286/391 293/391 =============================================>...............]  Step: 209ms | Tot: 1m1s | Loss: 0.001 | Acc: 99.984% (37882/3788 296/391 ===================================>...............]  Step: 209ms | Tot: 1m1s | Loss: 0.001 | Acc: 99.984% (38266/3827 299/391 =================================================>...............]  Step: 209ms | Tot: 1m2s | Loss: 0.001 | Acc: 99.984% (38394/3840 300/391 ================================================>..............]  Step: 202ms | Tot: 1m2s | Loss: 0.001 | Acc: 99.984% (38650/3865 302/391 ===================>..............]  Step: 

 [================================================================>]  Step: 138ms | Tot: 1m20s | Loss: 0.001 | Acc: 99.998% (49999/5000 391/391 1  =====================>.......................]  Step: 211ms | Tot: 51s745ms | Loss: 0.001 | Acc: 99.997% (31999/3200 250/391 ========================================>......................]  Step: 210ms | Tot: 52s989ms | Loss: 0.001 | Acc: 99.997% (32767/3276 256/391 ===================================>......................]  Step: 211ms | Tot: 53s407ms | Loss: 0.001 | Acc: 99.997% (33023/3302 258/39 260/391 ========================================>.....................]  Step: 213ms | Tot: 54s28ms | Loss: 0.001 | Acc: 99.997% (33407/3340 261/391 ===========================================>....................]  Step: 201ms | Tot: 55s474ms | Loss: 0.001 | Acc: 99.997% (34303/3430 268/391 ==========================>....................]  Step: 205ms | Tot: 55s892ms | Loss: 0.001 | Acc: 99.997% (34559/3456 270/391 ============================

 [================================================================>]  Step: 139ms | Tot: 1m20s | Loss: 0.001 | Acc: 99.986% (49993/5000 391/391 1  39/391 ================================>.........................]  Step: 199ms | Tot: 49s838ms | Loss: 0.001 | Acc: 99.987% (30844/3084 241/391 ========================================>.......................]  Step: 212ms | Tot: 51s719ms | Loss: 0.001 | Acc: 99.987% (31996/3200 250/391 ====================================>.......................]  Step: 203ms | Tot: 51s922ms | Loss: 0.001 | Acc: 99.988% (32124/3212 251/391 ============================>......................]  Step: 211ms | Tot: 52s553ms | Loss: 0.001 | Acc: 99.988% (32508/3251 254/391 =========================================>......................]  Step: 209ms | Tot: 53s175ms | Loss: 0.001 | Acc: 99.988% (32892/3289 257/391 ===========================>.....................]  Step: 210ms | Tot: 54s12ms | Loss: 0.001 | Acc: 99.988% (33404/3340 261/391 =====================

 [================================================================>]  Step: 139ms | Tot: 1m20s | Loss: 0.001 | Acc: 99.998% (49999/5000 391/391 ==============================>.....]  Step: 212ms | Tot: 1m14s | Loss: 0.001 | Acc: 99.998% (45823/4582 358/39 363/391 ================================================>...]  Step: 209ms | Tot: 1m16s | Loss: 0.001 | Acc: 99.998% (47359/4736 370/391 =============================================>...]  Step: 209ms | Tot: 1m17s | Loss: 0.001 | Acc: 99.998% (47615/4761 372/391 =====================================================>..]  Step: 199ms | Tot: 1m18s | Loss: 0.001 | Acc: 99.998% (48511/4851 379/391 =======================>.]  Step: 209ms | Tot: 1m19s | Loss: 0.001 | Acc: 99.998% (49023/4902 383/391 ===================================>.]  Step: 201ms | Tot: 1m19s | Loss: 0.001 | Acc: 99.998% (49151/4915 384/391 ================>.]  Step: 208ms | Tot: 1m19s | Loss: 0.001 | Acc: 99.998% (49279/4928 385/391 =====================================

 [================================================================>]  Step: 147ms | Tot: 1m20s | Loss: 0.001 | Acc: 99.988% (49994/5000 391/391 1  ======================================>...................]  Step: 216ms | Tot: 56s318ms | Loss: 0.001 | Acc: 99.989% (34812/3481 272/391 =>...................]  Step: 205ms | Tot: 56s728ms | Loss: 0.001 | Acc: 99.989% (35068/3507 274/391 ============================================>...................]  Step: 205ms | Tot: 57s353ms | Loss: 0.001 | Acc: 99.989% (35452/3545 277/391 ======================================>..................]  Step: 209ms | Tot: 58s187ms | Loss: 0.001 | Acc: 99.989% (35964/3596 281/391 ================>..................]  Step: 209ms | Tot: 58s397ms | Loss: 0.001 | Acc: 99.989% (36092/3609 282/391 ==========================================>.................]  Step: 210ms | Tot: 58s812ms | Loss: 0.001 | Acc: 99.989% (36348/3635 284/391 ===========================>.................]  Step: 201ms | Tot: 59s13ms | L